# EdX to Markdown format

See [here for a nice tutorial on beautifulsoup for xml files](http://www2.hawaii.edu/~takebaya/cent110/xml_parse/xml_parse.html).

In [1]:
from pathlib import Path
from bs4 import BeautifulSoup

## First read in a sample xml problem file

In [2]:
problem_file_name = 'test2_p8_vA.xml' # mcq
#problem_file_name = 'test2_p8_vC.xml' # numerical response

path = '../../edx_bank/library003/problem/' + problem_file_name


file = open(path,'r')
file_contents = file.read()

In [3]:
soup = BeautifulSoup(file_contents,'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<problem display_name="8.kappa" lg="2.200" markdown="null" max_attempts="1" rerandomize="per_student" showanswer="never">
<script type="text/javascript">
   (function() {
    $( ".check-label" ).text("Submit");
   })();
</script>
<script system_path="python_lib" type="text/python">
ga = 9.8
gb = 0.1*random.randint(40, 80)
a = 1.5
sol = ga+a-gb
c2 = ga+a+gb
c3 = ga+a
c4 = ga-gb
</script>
<p>8) During an inter-planetary experiment, a bathroom scale is placed on two elevators located in two different planets. Planet A (Earth) has a constant of gravity \(g_A = \) $ga m/s\(^2\) and planet B has a constant of gravity \(g_B = \) $gb m/s\(^2\). During the experiment, the elevator on Earth starts from rest and is accelerated upwards at a rate of \($a m/s^2\). If the scale reading for an astronaut during her way up in the elevator is the same in both planets, what is the magnitude of the elevator's acceleration (in \(m/s^2\)) on planet B? Note: the elevator

In [4]:
[tag.name for tag in soup.find_all()]

['problem',
 'script',
 'script',
 'p',
 'multiplechoiceresponse',
 'choicegroup',
 'choice',
 'choice',
 'choice',
 'choice',
 'demandhint',
 'hint']

### Title

In [5]:
title = soup.find_all('problem')[0].get('display_name')
title

'8.kappa'

### Author and Source

In [6]:
author = "edX Physics 100 Team"
source = "https://github.com/open-resources/edx_bank"

### Problem Text

In [25]:
problem_text = soup.find_all("p")[0].contents[0]

problem_text = (
    problem_text
    .replace("\(", "")  # get rid of left brackets
    .replace("\)", "")  # get rid of right brackets
    .replace("  $", " {{")  # replace edX way of using vars to new way: $ with {{. #TODO: Need to fix this by using regex to get the end }}.
    + "\n\n" # Add some new line chars
)

print(problem_text)

8) During an inter-planetary experiment, a bathroom scale is placed on two elevators located in two different planets. Planet A (Earth) has a constant of gravity g_A = {{ga m/s^2 and planet B has a constant of gravity g_B = {{gb m/s^2. During the experiment, the elevator on Earth starts from rest and is accelerated upwards at a rate of $a m/s^2. If the scale reading for an astronaut during her way up in the elevator is the same in both planets, what is the magnitude of the elevator's acceleration (in m/s^2) on planet B? Note: the elevator on planet B is also accelerating upwards. 




### Problem Type and Answer Choices (if applicable)

See all edX [problem types here](https://edx.readthedocs.io/projects/edx-partner-course-staff/en/latest/exercises_tools/create_exercises_and_tools.html).

In [33]:
problem_type = ""
answer_section = "## Answer Section\n\n"
mcq_python = ""

for tag in soup.find_all():

    if tag.name == "numericalresponse":
        problem_type = "numeric"

        answer_section += (
            f"The answer is: || { soup.find_all('numericalresponse')[0].get('answer') } &&."
            .replace("||", "{{")
            .replace("&&", "}}")
            .replace("$", "")
        )

    elif tag.name == "multiplechoiceresponse":
        problem_type = "mc"

        for i, response in enumerate(
            soup.find_all("multiplechoiceresponse")[0].find_all("choice")):

            answer_section += (
                f"- || { response.string.replace('$', '') } &&\n"
                .replace('||', "{{")
                .replace("&&", "}}")
            )

            mcq_python += f"data['params']['ans{ i+1 }'] = { response.string.replace('$', '') }\n"

            if response.attrs["correct"] == "true":
                mcq_python += f"data['params']['correct_answer'] = { response.string.replace('$', '') }\n"

    elif tag.name == "choiceresponse":
        problem_type = "ma"
    elif tag.name == "stringresponse":
        print("String response - plain text; not implemented yet")
        raise NotImplementedError
    elif tag.name == "optionresponse":
        print("Dropdown response; not implemented yet")
        raise NotImplementedError

if problem_type == "":
    raise NotImplementedError

### Python Solution/Code

In [34]:
python_code = Path('python_front.md').read_text()

for tag in soup.find_all('script'):
    
    if tag['type'] == 'text/javascript':
        pass
    elif tag['type'] == 'text/python':
        python_code += tag.contents[0]
        
full_python = python_code + Path('python_end.md').read_text() + mcq_python

print(full_python)

import random

# define or load names/items/objects

# store phrases etc

# define bounds of the variables

ga = 9.8
gb = 0.1*random.randint(40, 80)
a = 1.5
sol = ga+a-gb
c2 = ga+a+gb
c3 = ga+a
c4 = ga-gb

# store the variables in the dictionary params

# define possible answers

data['params']['ans1'] = sol
data['params']['correct_answer'] = sol
data['params']['ans2'] = c2
data['params']['ans3'] = c3
data['params']['ans4'] = c4



In [35]:
## Preparing the YAML dictionary

import yaml

# This solution is copied from this SO answer: https://stackoverflow.com/a/45004775/2217577 

yaml.SafeDumper.org_represent_str = yaml.SafeDumper.represent_str

def repr_str(dumper, data):
    if '\n' in data:
        return dumper.represent_scalar(u'tag:yaml.org,2002:str', data, style='|')
    return dumper.org_represent_str(data)

yaml.add_representer(str, repr_str, Dumper=yaml.SafeDumper)

In [36]:
yaml_dict = {}

yaml_dict['title'] = title
yaml_dict['author'] = author
yaml_dict['source'] = source
yaml_dict['type'] = problem_type
yaml_dict['tags'] = ['TBD']
yaml_dict['outcomes'] = ['TBD']
yaml_dict['assets'] = ['TBD']
yaml_dict['server'] = full_python #'import random \\n b=u'

In [37]:
print(yaml.safe_dump(yaml_dict,sort_keys=False))

title: 8.kappa
author: edX Physics 100 Team
source: https://github.com/open-resources/edx_bank
type: mc
tags:
- TBD
outcomes:
- TBD
assets:
- TBD
server: |
  import random

  # define or load names/items/objects

  # store phrases etc

  # define bounds of the variables

  ga = 9.8
  gb = 0.1*random.randint(40, 80)
  a = 1.5
  sol = ga+a-gb
  c2 = ga+a+gb
  c3 = ga+a
  c4 = ga-gb

  # store the variables in the dictionary params

  # define possible answers

  data['params']['ans1'] = sol
  data['params']['correct_answer'] = sol
  data['params']['ans2'] = c2
  data['params']['ans3'] = c3
  data['params']['ans4'] = c4



### Rest of the MD format

In [40]:
start = "# {{ vars.title }}\n\n## Question Text\n\n"


In [41]:
Path("test_question.md").write_text('---\n' +\
                                    yaml.safe_dump(yaml_dict,sort_keys=False) +\
                                    '---\n' +\
                                    start +\
                                    problem_text +\
                                    answer_section)

1324